In [1]:
import sys
sys.path.append('../scripts')
from scripts.cross_validation import loocv, k_fold_cv
from sklearn import ensemble
from scripts.data_preprocessing import load_data, exclude_outliers


# Load and preprocess data
X, y = load_data()

# Exclude outliers
X_filtered, y_filtered = exclude_outliers(X, y)

Number of outliers:  14


C:\Users\Tea\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


In [ ]:
# Define the parameter grid for Grid Search
# param_grid = {
#     "n_estimators": [100, 200, 500],
#     "max_depth": [3, 6, 9],
#     "max_leaf_nodes": [2, 6],
#     "learning_rate": [0.001, 0.01, 0.1, 1],
# }

params = {
    "n_estimators": 500,
    "max_depth": 4,
    "min_samples_split": 5,
    "learning_rate": 0.01,
    "loss": "squared_error",
}

# Initialize the model
gboost = ensemble.GradientBoostingRegressor(**params)

# Evaluate with LOOCV and k-fold
print('LOOCV')
loocv(X, y, gboost)

print('\n10-fold')
k_fold_cv(X, y, gboost, 6)

In [4]:
print('LOOCV')
loocv(X_filtered, y_filtered, gboost)

print('\n10-fold')
k_fold_cv(X_filtered, y_filtered, gboost, 6)

LOOCV
Average R^2 (LOOCV): nan
Average MSE (LOOCV): 36.1990337711179
Average RMSE (LOOCV): 3.679334123244461
Average MAE (LOOCV): 3.679334123244461

10-fold
Mean MSE: 43.649035461957
Mean MAE: 4.064879123544626
Mean R2: -0.5966198686068954
RMSE 6.60674166756632
